FDA Accelerated Approval Data

In [27]:
import tabula
import pandas as pd
import re

# Path to the PDF file
pdf_path = 'CDERAA.pdf'

# Extract tables from the PDF
tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, lattice=True)

# Combine all extracted tables into a single DataFrame
df_combined = pd.concat(tables, ignore_index=True)

PreProcessing

In [28]:
# Remove the first 3 rows
df_combined = df_combined.iloc[4:].reset_index(drop=True)

# Set the first row as the header
df_combined.columns = df_combined.iloc[0]
df_combined = df_combined[1:].reset_index(drop=True)

# Remove extra spaces/double spaces
df_combined.columns = df_combined.columns.str.replace(r'\s+', ' ', regex=True)


In [29]:
# Drop rows that contain NaN in Established Name
# Established name should always have a value in this list. 
# This provides an accurate way of knowing which rows are junk text from the PDF
df_combined = df_combined.dropna(subset=['Established Name'])

In [30]:
# Split Application Number and Supplement Number
# Regex \d means to 'match any digit'
df_combined['Supplement'] = df_combined['Application Number'].str.extract(r'(Supplement \d+)', expand=False)

# Rename the 'Supplement' column to 'Supplement'
# df_combined.rename(columns={'Supplement': 'Supplement'}, inplace=True)

# Extract numbers into a new column 'Supplement Number'
df_combined['Supplement Number'] = df_combined['Supplement'].str.extract(r'(\d+)', expand=False)

# Replace 'Supplement' with 'SUPPL' in the 'Supplement' column and remove numbers
df_combined['Supplement'] = df_combined['Supplement'].str.replace(r'Supplement \d+', 'SUPPL', regex=True).str.strip()

# Clean the original "Application Number" column to remove the "Supplement" part
df_combined['Application Number'] = df_combined['Application Number'].str.replace(r'\s*Supplement \d+', '', regex=True)

# Take out Original # and put it in its own column. Then remove Original from Application Number column
df_combined['Original'] = df_combined['Application Number'].str.extract(r'(Original \d+)', expand=False)
df_combined['Application Number'] = df_combined['Application Number'].str.replace(r'\s*Original \d+', '', regex=True)

# Replace 'Original' with 'ORIG' in the specified column
df_combined['Original'] = df_combined['Original'].str.replace(r'Original', 'ORIG', regex=True)

# Remove non-integer characters and convert to integers
# Regex: \D identifies any character not a digit
# df_combined['Total Time to Accelerated Approval (Months)'] = df_combined['Total Time to Accelerated Approval (Months)'].str.replace(r'\D', '', regex=True)
df_combined['Total Time to Accelerated Approval (Months)'] = df_combined['Total Time to Accelerated Approval (Months)'].astype(str).str.replace('†', '')

# The column contains dates that have extra numbers, making it invalid
# This code ensures the dates are in a date format and removes extra numbers
df_combined['Full Approval Conversion- Withdrawal Date'] = df_combined['Full Approval Conversion- Withdrawal Date'].astype(str).str.split().str[0]

# Check for # of NaN/NaT
# numofnans = df_combined['Full Approval Conversion- Withdrawal Date'].isna().sum()
# print(numofnans)

# Convert to datetime, coerce errors to Not a Time
# Removes any NaN/NaT
df_combined['Full Approval Conversion- Withdrawal Date'] = pd.to_datetime(df_combined['Full Approval Conversion- Withdrawal Date'], errors='coerce')

# Check again for of NaN just to make sure we didn't remove things that shouldn't have
numofnans = df_combined['Full Approval Conversion- Withdrawal Date'].isna().sum()
# print(numofnans)


In [31]:
# Some App #s have a superscript. Let's remove it.

def remove_appnum_hangingdigit(row):
    if row['Proprietary Name'] == 'KEYTRUDA' and row['Application Number'].endswith(' 4'):
        return row['Application Number'][:-1]
    elif row['Proprietary Name'] == 'RUBRACA' and row['Application Number'].endswith(' 5'):
        return row['Application Number'][:-1]
    return row['Application Number']


# Apply the function to the DF
df_combined['Application Number'] = df_combined.apply(remove_appnum_hangingdigit, axis=1)

In [32]:

# Change column dtypes
df_combined['Application Number'] = df_combined['Application Number'].astype("string").str.strip()
df_combined['Proprietary Name'] = df_combined['Proprietary Name'].astype("string")
df_combined['Established Name'] = df_combined['Established Name'].astype('string')
df_combined['Applicant'] = df_combined['Applicant'].astype("string")
df_combined['FDA Received Date'] = pd.to_datetime(df_combined['FDA Received Date'])
df_combined['Accelerated Approval Date'] = pd.to_datetime(df_combined['Accelerated Approval Date'])
df_combined['Total Time to Accelerated Approval (Months)'] = df_combined['Total Time to Accelerated Approval (Months)'].astype("float")
df_combined['Accelerated Approval Indication'] = df_combined['Accelerated Approval Indication'].astype("string")
df_combined['Conversion-Withdrawal Status'] = df_combined['Conversion-Withdrawal Status'].astype("string")
df_combined['Full Approval Conversion- Withdrawal Date'] = pd.to_datetime(df_combined['Full Approval Conversion- Withdrawal Date'])
df_combined['Supplement Number'] = df_combined['Supplement Number'].astype("string").fillna('').str.strip()
df_combined['Supplement'] = df_combined['Supplement'].astype("string").fillna('').str.strip()
df_combined['Original'] = df_combined['Original'].astype("string").fillna('').str.strip()

# Define desired order
desired_order = ['Application Number', 'Supplement', 'Supplement Number', 'Original', 'Proprietary Name', 'Established Name', 'Applicant', 'FDA Received Date', 'Accelerated Approval Date', 'Total Time to Accelerated Approval (Months)', 'Accelerated Approval Indication', 'Conversion-Withdrawal Status', 'Full Approval Conversion- Withdrawal Date']

# Re-index the dataframe
df_combined = df_combined[desired_order]

In [33]:
# Replace NaN in 'Total Time to Accelerated Approval (Months)' with None (null)
df_combined['Total Time to Accelerated Approval (Months)'] = df_combined['Total Time to Accelerated Approval (Months)'].where(pd.notnull(df_combined['Total Time to Accelerated Approval (Months)']), None)


In [34]:
# Function to categorize indication
def categorize_indication(indication):
    indication_lower = str(indication).lower()
    if 'adult' in indication_lower and 'pediatric' in indication_lower:
        return 'Both Adult and Pediatric'
    elif 'adult' in indication_lower:
        return 'Adults Only'
    elif 'pediatric' in indication_lower:
        return 'Pediatric Only'
    else:
        return 'Unspecified'

# Apply the function to the df_combined DataFrame
df_combined['Category'] = df_combined['Accelerated Approval Indication'].apply(categorize_indication)

# Display the updated DataFrame
df_combined.head()


,Application Number,Supplement,Supplement Number,Original,Proprietary Name,Established Name,Applicant,FDA Received Date,Accelerated Approval Date,Total Time to Accelerated Approval (Months),Accelerated Approval Indication,Conversion-Withdrawal Status,Full Approval Conversion- Withdrawal Date,Category
0,NDA 203469,SUPPL,37,,ICLUSIG,PONATINIB,TAKEDA PHARMACEUTICALS USA INC,2023-09-21,2024-03-19,5.9,FOR THE TREATMENT OF ADULT PATIENTS WITH NEWLY...,Not Yet Converted,NaT,Adults Only
1,NDA 217785,,,,REZDIFFRA,RESMETIROM,MADRIGAL PHARMACEUTICALS INC,2023-07-14,2024-03-14,8.0,"IN CONJUNCTION WITH DIET AND EXERCISE, FOR THE...",Not Yet Converted,NaT,Adults Only
2,NDA 213217,SUPPL,11,,BRUKINSA,ZANUBRUTINIB,BEIGENE USA INC,2023-05-11,2024-03-07,9.9,FOR THE TREATMENT OF ADULT PATIENTS WITH RELAP...,Not Yet Converted,NaT,Adults Only
3,NDA 216059,SUPPL,1,,JAYPIRCA,PIRTOBRUTINIB,LOXO ONCOLOGY INC,2023-06-02,2023-12-01,6.0,FOR THE TREATMENT OF ADULT PATIENTS WITH CHRON...,Not Yet Converted,NaT,Adults Only
4,BLA 125514,SUPPL,148,,KEYTRUDA,PEMBROLIZUMAB,MERCK SHARP & DOHME LLC,2023-08-04,2023-11-07,3.1,"IN COMBINATION WITH TRASTUZUMAB, FLUOROPYRIMID...",Not Yet Converted,NaT,Unspecified


In [35]:
# Replace NaN values with an empty string
df_combined['Original'] = df_combined['Original'].fillna('')

# Strip whitespace from the new column
df_combined['Original'] = df_combined['Original'].str.strip()

In [36]:
# Replace blank or empty values in the 'Original' column with 'ORIG 1' only if 'Supplement' contains a space
df_combined.loc[df_combined['Supplement'] == '', 'Original'] = df_combined.loc[df_combined['Supplement'] == '', 'Original'].replace('', 'ORIG 1')
df_combined['Original'] = df_combined['Original'].str.strip()

In [37]:
# CREATE new column combining App # + Suppl/ORIG + Suppl # in DF_COMBINED

df_combined['Full_App_Supp_Number'] = df_combined.apply(
    lambda row: f"{row['Application Number']} {row['Supplement']} {row['Supplement Number']}", axis=1
)

df_combined['Full_App_Supp_Number'] = df_combined.apply(
    lambda row: f"{row['Full_App_Supp_Number']} {row['Original']}", axis=1
)

# Replace NaN values with an empty string
df_combined['Full_App_Supp_Number'] = df_combined['Full_App_Supp_Number'].fillna('')

# Strip whitespace from the new column
df_combined['Full_App_Supp_Number'] = df_combined['Full_App_Supp_Number'].str.strip()

In [38]:
# Remove double spaces in Full App Supp Number
df_combined['Full_App_Supp_Number'] = df_combined['Full_App_Supp_Number'].apply(lambda x: re.sub(' +', ' ', x))

OpeFDA Data 

In [39]:
# Import the openFDA parquet file

import pandas as pd

parquet_file = 'openfda.parquet'

df_main = pd.read_parquet(parquet_file)

# print(df.head())

In [40]:
# CREATE new column combining App # + Suppl + Suppl # in DF_MERGED, the OpenFDA data
# Giives UNIQUE IDs that wiill match up to the AA data set
df_main['Full_App_Supp_Number'] = df_main.apply(
    lambda row: f"{row['application_number_with_space']} {row['submissions_submission_type']} {row['submissions_submission_number']}", axis=1
)
df_main['Full_App_Supp_Number'] = df_main['Full_App_Supp_Number'].apply(lambda x: re.sub(' +', ' ', x))

In [41]:
# Remove duplicates in the 'Full_App_Supp_Number' column, keeping only the first occurrence
df_main_unique = df_main.drop_duplicates(subset=['Full_App_Supp_Number'], keep='first')


In [42]:
# Convert Open_FDA_route_x to a string
df_main_unique['openfda_route_x'] = df_main_unique['openfda_route_x'].astype("string").str.strip()


C:\Users\Foxhound\AppData\Local\Temp\ipykernel_35932\2539915300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main_unique['openfda_route_x'] = df_main_unique['openfda_route_x'].astype("string").str.strip()


In [43]:
# CREATE NEW Merged DF
# Adds OpenFDA ROUTE

df_combined2 = df_combined.merge(df_main_unique[['Full_App_Supp_Number', 'openfda_route_x', 'submissions_submission_class_code_description', 'submissions_submission_property_type', 'products_te_code', 'products_marketing_status', 'submissions_review_priority']], 
                                 on='Full_App_Supp_Number', 
                                 how='left')

In [44]:
# CLEAN MISSING VALUES IN DF_COMBINED2
df_combined2.loc[df_combined2['Application Number'] == 'NDA 214938', 'openfda_route_x'] = 'SUBCUTANEOUS'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 761137', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 216387', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 202806', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 761139', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 215935', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 761223', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 214622', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 761115', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 125557', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 214383', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 213176', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 761145', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 214701', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 761158', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 202306', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 213400', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 212306', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 212726', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 761121', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 211155', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 761078', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 205353', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 206162', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 206256', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 205755', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 203585', 'openfda_route_x'] = 'SUBCUTANEOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 202497', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 022393', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021945', 'openfda_route_x'] = 'INTRAMUSCULAR'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 022273', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 022291', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020634', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021721', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021430', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021968', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 125011', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021322', 'openfda_route_x'] = 'SUBCUTANEOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021677', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021335', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021399', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 103979', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021481', 'openfda_route_x'] = 'SUBCUTANEOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021335', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021492', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021226', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 019858', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020780', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 019847', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 019857', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 761060', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021156', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 050747', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021007', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021041', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 103767', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020977', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020972', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020636', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 019832', 'openfda_route_x'] = 'TOPICAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020896', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020705', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020778', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 019815', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020604', 'openfda_route_x'] = 'SUBCUTANEOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020636', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020685', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020659', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020680', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020628', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020212', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020412', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 050697', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 050698', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 103471', 'openfda_route_x'] = 'SUBCUTANEOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020199', 'openfda_route_x'] = 'ORAL'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 761208', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 761038', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 020635', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'BLA 125019', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 019537', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021174', 'openfda_route_x'] = 'INTRAVENOUS'
df_combined2.loc[df_combined2['Application Number'] == 'NDA 021039', 'openfda_route_x'] = 'ORAL'

# Remove single quotes and brackets
df_combined2['openfda_route_x'] = df_combined2['openfda_route_x'].str.replace(r"[\'\[\]]", "", regex=True).str.strip()


Combine Drug Company list and PDUFA fees list

In [45]:
# LOAD DRUG COMPANY AND COUNTRY LIST

file_path = 'drugcompanycountries.xlsx' 
df_drugcompanycountries = pd.read_excel(file_path)

df_combined2['Applicant'] = df_combined2['Applicant'].astype("string").str.replace('\n', ' ').str.replace('\r', ' ').str.strip()

# Merge to df_combined2 based on Applicant 
df_combined2_merged = pd.merge(df_combined2, df_drugcompanycountries[['Applicant', 'Country']], on='Applicant', how='left')

In [46]:
# Convert FDA Received Date to datetime. Then save just the year to RecvDate

df_combined2_merged['FDA Received Date'] = pd.to_datetime(df_combined2_merged['FDA Received Date'])

df_combined2_merged['RecvDate'] = df_combined2_merged['FDA Received Date'].dt.year.astype(int)

# Load PDUFA Fees list

file_path = 'PDUFAFees.xlsx' 
df_PDUFAFees = pd.read_excel(file_path)

In [47]:
# Calculate time to conversion
df_combined2_merged['Total Time to Conversion (Months)'] = ((df_combined2_merged['Full Approval Conversion- Withdrawal Date'] - df_combined2_merged['Accelerated Approval Date']).dt.days / 30).round()

In [48]:
# Merge PDUFA Fees list on Year
df_combined2_merged2 = pd.merge(df_combined2_merged, df_PDUFAFees[['Year', 'Fees']], left_on='RecvDate', right_on='Year', how='left')

In [49]:
# Drop the Year and RecvDate columns
df_combined2_merged2.drop('Year', axis=1, inplace=True)

In [50]:
# Convert dates and fees according to Fiscal Year instead of Calendar Year

# Define function for Fiscal Year
def get_fiscal_year(date):
        if date.month >= 10:
                return date.year + 1
        else:
                return date.year
        
# Apply the function to the received date column, and save as Fiscal Year in the RecvDate Column
df_combined2_merged2['RecvDate'] = df_combined2_merged2['FDA Received Date'].apply(get_fiscal_year)

In [51]:
# Create a dictionary from df_PDUFAFees for quick lookup
# Testing out how to use dictionaries
fees_dict = df_PDUFAFees.set_index('Year')['SupplementFees'].to_dict()

# Update the 'Fees' column with the supplement fees for each respective year, if the submission is an Efficacy supplement.
df_combined2_merged2.loc[
    (df_combined2_merged2['submissions_submission_class_code_description'] == 'Efficacy') &
    (df_combined2_merged2['RecvDate'] < 2018),
    'Fees'
] = df_combined2_merged2['RecvDate'].map(fees_dict)

In [52]:
# Update the 'Fees' column based on PDUFA VI no longer collecting fees for efficacy supplements
df_combined2_merged2.loc[
    (df_combined2_merged2['submissions_submission_class_code_description'] == 'Efficacy') &
    (df_combined2_merged2['RecvDate'].isin([2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027])),
    'Fees'
] = 0

Export Dataframe

In [53]:
output = 'accelerated_approvals_vis.xlsx'
df_combined2_merged2.to_excel(output, index=False)
print(f"Dataframe has been exported to {output}")

Dataframe has been exported to accelerated_approvals_vis.xlsx
